In [1]:
import pandas as pd
import numpy as np
import time
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm

In [2]:
DRIVER_PATH = '/path/to/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

C:\Users\Shantanu Shetty\AppData\Local\Temp\ipykernel_16324\1326500500.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [3]:
file = pd.read_csv('data/EDA_df.txt.gz', sep='\t', compression='gzip')
file.head(2)

,SureChEMBL_ID,SMILES,InChIKey,PATENT_ID,PUBLICATION_DATE,Field
0,SCHEMBL4,C[C@H](CS)C(=O)N1CCC[C@H]1C(O)=O,FAKRSMQSSFJEIM-RQJHMYQMSA-N,EP-2842582-A2,2015-03-04,Description
1,SCHEMBL4,C[C@H](CS)C(=O)N1CCC[C@H]1C(O)=O,FAKRSMQSSFJEIM-RQJHMYQMSA-N,EP-2838373-A2,2015-02-25,Description


In [4]:
unique_patent_id = file.drop_duplicates(subset='PATENT_ID', keep="first")
unique_patent_id.reset_index(drop=True, inplace=True)

In [5]:
unique_patent_id['PATENT_ID'] = unique_patent_id['PATENT_ID'].str.replace('-', '')

C:\Users\Shantanu Shetty\AppData\Local\Temp\ipykernel_16324\2045907726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_patent_id['PATENT_ID'] = unique_patent_id['PATENT_ID'].str.replace('-', '')


In [ ]:
if os.path.exists('PN_CODE.tsv'):
    ko_df = pd.read_csv('PN_CODE.tsv', sep='\t')
else:
    ko_df = pd.DataFrame(columns=['PATENT_ID', 'PATENT_CODE'])

link = 'https://www.surechembl.org/search/'

c = 0
pncode_list = [] 

for index, row in tqdm(unique_patent_id.iterrows(), total=unique_patent_id.shape[0]):
    
    pn_cd = unique_patent_id.iloc[index]['PATENT_ID']

    if pn_cd in ko_df['PATENT_ID'].values.tolist():
        continue

    driver.get(link)
    time.sleep(5)
    
    c += 1
    try:
        SEARCH_CODE = driver.find_element(by=By.XPATH, value='/html/body/div/div/div[2]/div/div[1]/div/form/div[2]/div[2]/div[1]/textarea')
        CODE = 'pn:' + pn_cd
        SEARCH_CODE.send_keys(CODE)
        SEARCH_button = driver.find_element(by=By.XPATH, value='/html/body/div/div/div[2]/div/div[1]/div/form/div[2]/div[3]/div[1]/div/input')
        SEARCH_button.click()
        time.sleep(10)
    

        PATENT_CODE = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div[2]/table/tbody/tr[2]/td[4]/div[1]/table/tbody/tr/td[1]').text

        pncode_list.append({'PATENT_ID': pn_cd, 'PATENT_CODE': PATENT_CODE})
        
        driver.get(link)
        
    except NoSuchElementException:
        pncode_list.append({'PATENT_ID': pn_cd, 'PATENT_CODE': 'NOT FOUND'})
        driver.get(link)
    
    if c == 10 or index == unique_patent_id.shape[0] - 1:
        kemp_df = pd.DataFrame(pncode_list)
        ko_df  = pd.concat([ko_df, kemp_df], ignore_index=True)
        ko_df.to_csv('PN_CODE.tsv', index=False, sep='\t')
        c = 0
        pncode_list = []
    
driver.quit()

  1%|          | 18154/1574602 [2:07:39<6781:31:07, 15.69s/it]